In [1]:
%load_ext autoreload
%autoreload 2

import drukarnia_api

Слава Україні 🇺🇦


### author

In [2]:
# create an author
author = drukarnia_api.Author(username='drukarnia')

In [3]:
# Context manager creates an aiohttp session
async with author:
    # Password and email are not stored. Only retrievable data is cookie strings.
    await author.login(email='pantry.gene.0y@icloud.com', password='topciz-3haxPo-mezvuz')

In [4]:
async with author:
    follower, *_ = await author.get_followers(n_collect=1)

    # collect more data about the user
    await follower.collect_data()

    print('my follower\'s name is', await follower.name)
    print('my follower was created at', await follower.created_at)
    print('am I subscribed to my follower? -', await follower.is_subscribed)

my follower's name is Читач🍌
my follower was created at 2023-04-15 12:28:04.246000
am I subscribed to my follower? - False


In [5]:
async with follower:
    await follower.subscribe(unsubscribe=True)

    # to be 100% sure we will request new data
    await follower.collect_data()

    print('am I subscribed to my follower now? -', await follower.is_subscribed)

am I subscribed to my follower now? - False


In [6]:
async with follower:
    fol1, *_ = await follower.get_followings(n_collect=5)
    print(await fol1.name)

ブラックライト


In [7]:
async with follower:
    # collect all followings
    all_followings = await follower.get_followings()

    print(len(all_followings))

# results an error
"""
DrukarniaAPIError:
        Request Type: GET
        Response Status: 429
        Request To: https://drukarnia.com.ua/api/relationships/643a985486518bfbf4936848/following?page=30
        Error Message: "ThrottlerException: Too Many Requests"
"""

DrukarniaAPIError: 
        Request Type: GET
        Response Status: 429
        Request To: https://drukarnia.com.ua/api/relationships/643a985486518bfbf4936848/following?page=30
        Error Message: "ThrottlerException: Too Many Requests"
        

In [19]:
async with follower:
    ten_followers = await follower.get_followers(offset=0, n_collect=10)

    print([await user.username for user in ten_followers])

['Serdolyk', 'Av_lyu', 'H0pToad', 'anSk', 'Exemplerwriter', 'per_nat', 'rostifan', 'Spivanuk93', 'Rodj', 'Anton']


### Article

In [36]:
async with follower:
    articles = await follower.articles
    print(f'Author {await follower.name} has {len(articles)} articles')

Author Читач🍌 has 1 articles


In [37]:
article, = articles

In [38]:
async with article:
    await article.collect_data()

In [39]:
print(await article.read_time)
print(await article.number_of_like)
print(await article.title)

48.0
617
Кого я ще не читаю?


In [44]:
async with article:
    print(await article.my_likes)

    await article.like_article(5)
    await article.collect_data()

    print(await article.my_likes)

7
5


In [45]:
async with article:
    await article.post_comment('😝')
    print('posted comment')

posted comment


### Comment

In [51]:
async with article:
    await article.collect_data()

    # ONLY tree heads, replies are obtained from comment object itself
    comments = await article.comments

    print(f'article {await article.seo_title} has {len(comments)} comments')

article Кого я ще не читаю? has 47 comments


In [52]:
comment, *_ = comments

In [57]:
{
'text': await comment.text,
'number of likes': await comment.number_of_likes,
'number of replies': await comment.number_of_replies,
'have I liked it': await comment.is_liked,
'owner\'s name': await (await comment.owner).name
}

{'text': '<p>я в вашій чарівній колекції вже є, але тепер Я ПІДПИСУЮСЬ НА ВАС, ЩОБ МАТИ ДОСТУП ДО <strong>УСІХ</strong> АВТОРІВ НА ЦІЙ ПЛАТФОРМІ, МУХАХАХАХХАА! </p>',
 'number of likes': 18,
 'number of replies': 3,
 'have I liked it': False,
 "owner's name": 'сироманка🧀'}

In [61]:
async with comment:
    replies = await comment.get_replies()
    print(f'comment has {len(replies)} replies')

comment has 3 replies


### Tag

In [63]:
async with article:
    tags = await article.article_tags

    print(f'article {await article.title} has {len(tags)} tags')

article Кого я ще не читаю? has 5 tags


In [64]:
tag, *_ = tags

In [65]:
print(f'tag {await tag.slug} was mentioned {await tag.mentions_num}')

tag ukrayina was mentioned 439


In [66]:
async with tag:
    related = await tag.related_tags()

    print(f'retrieved {len(related)} related tags to tag {await tag.slug}')

retrieved 6 related tags to tag ukrayina


In [73]:
[await t.name for t in related]

['Війна', 'Політика', 'Українізація', 'Історія', 'Культура', 'Література']

### How aiohttp sessions are managed?

#### Automatic Session Management:

> After creating an object:
> ```py
> author = drukarnia_api.Author(username='drukarnia')
> ```
> ##### Headers (drukarnia_api.Headers) and cookies (drukarnia_api.DrukarniaCookies) are instantly created. However, the session will be created/closed only inside the context manager (__aenter__() and __aexit__()):
> ```py
> async with author:
>    follower, *others = await author.get_followers(n_collect=1)
>    print(await follower.name)
> ```
> ##### All drukarnia_api objects (e.g., follower ~ drukarnia_api.Author) created inside the context manager will inherit the manager's (in this context, author) session and credentials. This means that inside the same context window, all created objects can make requests without any additional context managers, with the same authentication cookies. Once the context manager is exited, the session will be closed, but credentials will be saved. Example:
> ```py
> async with author:
>     follower, *others = await author.get_followers(n_collect=1)
>
>     # Subscribe to your follower from the author's account
>     await follower.subscribe()
>
> async with follower:
>     # Unsubscribe to your follower from the author's account
>     await follower.subscribe(unsubscribe=True)
> ```
> ##### Note that this logic is also applicable for properties.
> ##### Example | (credentials shared):

> ```py
> async with author:
>     article, *_ = await author.articles
>
> async with article:
>     await article.like_article(1)
> ```
> ##### Example || (credentials NOT shared):

> ```py
> article, *_ = await author.articles
>
> async with article:
>     # Error: 'User needs to sign up first'
>     await article.like_article(1)
> ```

#### Manual Session Management:

> ##### You can create your own session and pass it to the object. In this case, you don't need to use a context manager; however, it's your responsibility to close the session. Example:

> ```py
> import drukarnia_api
> import aiohttp
>
> custom_session = aiohttp.ClientSession(
>     base_url=drukarnia_api.Connection.base_url,
>     # cookies=drukarnia_api.DrukarniaCookies(),
>     # headers=...
> )
>
> author = drukarnia_api.Author(username='drukarnia', session=custom_session)
>
> # Follower's session == custom_session
> follower, *others = await author.get_followers(n_collect=1)
>
> await custom_session.close()
> ```